In [9]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [11]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md


In [12]:
ls data

shoes_prices.csv  shoes_prices_usd.csv


In [0]:
df = pd.read_csv("data/shoes_prices_usd.csv", low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=500)):
  X = df[feats].values
  y = df.prices_amountmin.values

  scores = cross_val_score(model, X, y, scoring= "neg_mean_absolute_error")
  return(np.mean(scores), np.std(scores))



In [105]:
df["brand_cat"] = df["brand"].factorize()[0]

run_model(["brand_cat"])

(-46.11320836375886, 3.3305727012666955)

In [103]:
model = RandomForestRegressor(max_depth=500, n_estimators=100, random_state = 0)
run_model(["brand_cat"], model)

(-45.7996913742448, 3.24886905887665)

In [21]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [106]:
df["brand_cat"] = df["brand"].map(lambda x: str(x).lower()).factorize()[0]

run_model(["brand_cat"])

(-45.84410217248452, 3.216343160242935)

In [108]:
model = RandomForestRegressor(max_depth=500, n_estimators=100, random_state = 0)
run_model(["brand_cat"], model)

(-45.7996913742448, 3.24886905887665)

In [28]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [30]:
test = {"key": "value"}
test["key"]

'value'

In [31]:
str(test)

"{'key': 'value'}"

In [0]:
def parse_features(x):
  if str(x) == "nan": return []
  return literal_eval(x.replace('\\"', '"'))
  

df["features_parsed"] = df["features"].map(parse_features)

In [43]:
df.features_parsed.head()

0    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
1    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
2    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
3    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
4    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
Name: features_parsed, dtype: object

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == "nan": return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item["key"].lower().strip()
    value = item["value"][0].lower().strip()

    output_dict[key] = value

  return output_dict  

df["features_parsed"] = df["features"].map(parse_features)

In [113]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [114]:
keys = set()

df.features_parsed.map(lambda x: keys.update(x.keys()))
len(keys)


476

In [115]:
def get_name_feat(key):
  return "feat_" + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [116]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_pattern', 'feat_frame shape', 'feat_sports league', 'feat_front',
       'feat_sizebridge', 'feat_brand_cat', 'feat_color_cat',
       'feat_gender_cat', 'feat_manufacturer_cat', 'feat_material_cat'],
      dtype='object', length=531)

In [0]:
keys_stat = {}
for key in keys:
 keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] *100

In [118]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [119]:
{k:v for k, v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df["feat_brand_cat"] = df["feat_brand"].factorize()[0]
df["feat_color_cat"] = df["feat_color"].factorize()[0]
df["feat_gender_cat"] = df["feat_gender"].factorize()[0]
df["feat_manufacturer_cat"] = df["feat_manufacturer"].factorize()[0]
df["feat_material_cat"] = df["feat_material"].factorize()[0]



In [131]:
df["brand"] = df["brand"].map(lambda x: str(x).lower())
df[df.brand != df.feat_brand][["brand", "feat_brand"]].head()


,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [122]:
df[df.brand == df.feat_brand].shape

(52, 531)

In [123]:
feats = ["brand_cat", 
         "feat_color_cat",
         "feat_gender_cat", 
         "feat_manufacturer_cat", 
         "feat_material_cat"]
         
model = RandomForestRegressor(max_depth=1000, n_estimators=100)
run_model(feats, model)

(-46.92233067545254, 3.7146483062433218)

In [125]:
feats_2 = ["brand_cat", 
         "feat_color_cat",
         "feat_gender_cat", 
         "feat_material_cat"]
model_2 = RandomForestRegressor(max_depth=1000, n_estimators=100)
run_model(feats_2, model_2)


(-46.88577628619049, 3.623747670075486)

In [127]:
feats_3 = ["brand_cat",  
         "feat_material_cat"]
model_3 = RandomForestRegressor(max_depth=300, n_estimators=100)
run_model(feats_3, model_3)

(-47.06153423481703, 3.4487622004716463)

In [141]:
X = df[feats].values
y = df.prices_amountmin.values

m = RandomForestRegressor(max_depth=300, n_estimators=100, random_state = 0)
m.fit(X, y)

print(run_model(feats, model))
perm = PermutationImportance(m, random_state=1).fit(X, y);
eli5.show_weights(perm, feature_names = feats)


(-46.906061264288624, 3.7403598469713266)


Weight,Feature
0.9870 ± 0.0203,brand_cat
0.3211 ± 0.0205,feat_material_cat
0.1782 ± 0.0066,feat_color_cat
0.1580 ± 0.0071,feat_gender_cat
0.0015 ± 0.0001,feat_manufacturer_cat


In [98]:
|df.brand.value_counts(normalize = True)

Nike                        0.093577
PUMA                        0.031430
Ralph Lauren                0.028692
New Balance                 0.019767
VANS                        0.019493
                              ...   
Cobian                      0.000055
IB Pad                      0.000055
Cudas                       0.000055
Adidas Crazy 8 NY Knicks    0.000055
Kangaroo                    0.000055
Name: brand, Length: 1856, dtype: float64

In [99]:
df[df["brand"] == "nike"].features_parsed.head().values

array([{'style': 'Running, Cross Training', 'material': 'Synthetic', 'condition': 'New with box'},
       {'sport': 'Baseball & Softball', 'type': 'Cleats', 'condition': 'New with box'},
       {'condition': 'New with box', 'type': 'Cleats'},
       {'style': 'Running, Cross Training, Walking', 'condition': 'New without box'},
       {'sport': 'Soccer', 'type': 'Cleats'}], dtype=object)

Najlepszy uzyskany jak dotąd wynik, to wynik dla modelu *bazowego* (z samym brandem i odpowiednim *ziarnem losowania*). Jedynym powodem tego, że uzyskane wyniki są trochę lepsze niż na filmie, jest fakt, że w Random Forest (w przeciwieństwie do boostingu) nie powinno się przycinać drzew, ale pozwolić na to, żeby drzewa były w miarę możliwości jak największe. Widać, że to jest ważne także przy badaniu ważności cech.

In [147]:
model_a = RandomForestRegressor(max_depth=300, n_estimators=200, random_state = 20)
run_model(["brand_cat"], model_a)

(-45.68722491877723, 3.237184423515146)

Dla porównania - model max_depth = 5:

In [148]:
model_b = RandomForestRegressor(max_depth=5, n_estimators=200, random_state = 20)
run_model(["brand_cat"], model_b)

(-57.337951368166195, 4.138543756405692)